# Analysis of wage and hour case data from US state labor agencies

## Data import and preparation

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [2]:
df = pd.read_csv(
    "input/state_complaints.csv.gz",
    low_memory=False,
    parse_dates=["date_opened", "date_closed", "date_paid"],
)


## Assignment of helper columns

In [3]:
# TODO: add notes
notna_and_nonzero = lambda val: pd.notna(val) and val != 0

df = df.assign(
    # number of days between either paid or closed date and opened date
    case_duration=lambda df: (
        df.apply(
            lambda row: row.date_paid - row.date_opened
            if pd.notna(row.date_paid)
            else row.date_closed - row.date_opened,
            axis=1,
        ).apply(lambda val: val.days)
    ),
).assign(_has_case_duration=lambda df: df.case_duration.notna())


### Open, dismissed, withdrawn and other cases

Some states provided case statuses that indicated the outcome of the case. Some states provided case statuses that indicated only "closed" or "open" and no more specifics. Others provided no case status at all. In the transform task, I converted the statuses provided into the categories below. Each agency provided definitions of their statuses prior to this, unless otherwise documented. 

Below I will remove case statuses that indicate the case was rejected or withdrawn, or is still open. For states that didn't provide provide statuses, it is my understanding that they only contain closed cases and that open or rejected cases were not provided, so this should make this data as consistent across agencies as possible.

In [4]:
print(df.case_status.unique())


[nan 'closed' 'overturned' 'pending appeal' 'affirmed' 'open' 'dismissed'
 'pending enforcement' 'withdrawn' 'amount exceeds statutory limit']


In [5]:
states_with_status = df.query("case_status.notna()").state_name.unique()
df = df.assign(
    _case_complete=lambda df: (
        df.apply(
            lambda row: (
                (
                    row.state_name in states_with_status
                    and (
                        pd.isna(row.case_status)
                        or row.case_status
                        in [
                            "closed",
                            "pending enforcement",
                            "affirmed",
                            "amount exceeds statutory limit",
                        ]
                    )
                )
                or (row.state_name not in states_with_status)
            ),
            axis=1,
        )
    )
)
df.groupby("_case_complete").size()


_case_complete
False     29530
True     564093
dtype: int64

### Removal of cases with no assessed amounts, if possible


In [6]:
# list of states that provided assessed amounts in their data
states_with_claimed_and_assessed = (
    df.groupby("state_name")[["amount_claimed", "amount_assessed"]]
    .apply(
        lambda gb: gb.amount_claimed.notna().any() and gb.amount_assessed.notna().any()
    )
    .to_frame("state_has_claimed_and_assessed")
    .query("state_has_claimed_and_assessed")
    .index
)

df = df.assign(
    _nonzero_assesed_amount_or_no_assessed_amount_provided=lambda df: df.apply(
        lambda row: (
            (
                row.state_name in states_with_claimed_and_assessed
                and row.amount_assessed > 0
            )
            or (row.state_name not in states_with_claimed_and_assessed)
        ),
        axis=1,
    )
)

df.groupby("_nonzero_assesed_amount_or_no_assessed_amount_provided").size()


_nonzero_assesed_amount_or_no_assessed_amount_provided
False    254035
True     339588
dtype: int64

### Multi-violation cases


In [7]:
# TODO: add notes
df = df.drop_duplicates(subset=["case_uuid"], keep="first")


### Data selection

In [8]:
def select_data(*flags):
    q = ""
    for i, flag in enumerate(flags):
        if i == len(flags) - 1:
            q += f"{flag}"
        else:
            q += f"{flag} and "

    return (
        # first do the query
        df.query(q)
        # the dedupe on the case uuid field so all available
        # categories are included
        .drop_duplicates(subset=["case_uuid"], keep="first").drop("case_uuid", axis=1)
    )


# Analysis

## Amounts

#### Loading separate Texas amount data

The Texas Workforce Commission claimed that the amounts claimed in each case were private financial information and refused to release them in a way that could be associated with the case information. I convinced them to send me a file containing *only* amounts. 

I therefore cannot remove closed cases like I did with the rest of this data, however in the interest of having *some* data from Texas, I will incorporate this to the rest of the amounts. 

**THIS IS SUBJECT TO CHANGE PENDING A RULING FROM THE TEXAS ATTORNEY GENERAL**

In [9]:
tx = (
    pd.read_excel("input/ORR_R005317-081222_from_CBS__C._Hacker__File_date___Amts.xlsx")
    .rename(
        columns={
            "FILG_DT": "filing_year",
            "CLAIMED": "amount_claimed",
            "AWARD_AM": "amount_assessed",
            "PAID": "amount_paid",
        }
    )
    # there are no null values, just 0s
    .query("amount_assessed != 0")
    # because I have no way of excluding closed or rejected cases, I h ave 
    .query("amount_claimed < @df.amount_claimed.quantile(0.99)")
    .assign(state_name="Texas")
)

def append_texas(dataframe):
    """helper function to append texas amount data to dataframe"""
    return pd.concat([dataframe.query("state_name != 'Texas'"), tx])



In [10]:
amt_df = (
    select_data(
        "_case_complete",
        "_nonzero_assesed_amount_or_no_assessed_amount_provided",
    )
    .pipe(append_texas)
    .assign(
        amount_claimed_or_assessed=lambda df: df.apply(
            # use assessed amount because it is the most accurate
            lambda row: row.amount_assessed if pd.notna(row.amount_assessed)
            # if not available, use the amount claimed
            else row.amount_claimed if pd.notna(row.amount_claimed)
            # only use amount paid if neither assessed nor claimed are available
            # because there are so many states taht don't accurately report
            # paid amounts
            else row.amount_paid if pd.notna(row.amount_paid) else np.NaN,
            axis=1,
        )
    )
)

amt_df.amount_claimed_or_assessed.describe()


count    276809.000
mean       5651.167
std       33581.529
min           0.000
25%         360.310
50%        1010.630
75%        3300.000
max     5912348.830
Name: amount_claimed_or_assessed, dtype: float64

In [11]:
(
    amt_df
    .groupby("state_name")
    .amount_claimed_or_assessed
    .describe()
)


,count,mean,std,min,25%,50%,75%,max
state_name,,,,,,,,
California,53754.000,10392.113,48424.754,0.070,864.000,3200.000,8400.548,5912348.830
Colorado,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Illinois,11910.000,4675.315,37815.497,0.000,286.000,960.000,2700.000,1923000.000
Indiana,4573.000,885.145,3623.796,2.000,212.310,490.000,1015.000,233373.000
Iowa,376.000,2271.008,1667.114,69.000,977.750,1903.000,3200.000,7625.000
Kansas,9251.000,3731.729,15532.825,0.440,390.085,904.120,2317.720,500000.000
Kentucky,7137.000,3340.072,16041.229,0.000,183.060,374.960,1150.000,125810.820
Maine,792.000,2496.591,8276.580,2.750,222.040,585.750,1551.312,153993.440
Maryland,8290.000,2857.271,8325.920,0.000,425.000,1068.990,2591.500,408655.000


## Case duration

In [12]:
cd_df = select_data(
    "_case_complete",
    "_nonzero_assesed_amount_or_no_assessed_amount_provided",
    "_has_case_duration",
).copy()
cd_df.case_duration.describe()


count   110428.000
mean       267.342
std        395.210
min          0.000
25%         76.000
50%        159.000
75%        297.000
max       4585.000
Name: case_duration, dtype: float64

In [13]:
(
    cd_df.groupby(pd.cut(bins=[0, 7, 28, 90, 180, 365, 730, 999999999], x=cd_df.case_duration))
    .size()
    .to_frame("total_cases")
    .assign(pct=lambda df: df.total_cases / df.total_cases.sum())
    .rename(
        index={
            pd.Interval(0, 7, closed="right"): "0-7 days",
            pd.Interval(7, 28, closed="right"): "7-28 days",
            pd.Interval(28, 90, closed="right"): "28-90 days",
            pd.Interval(90, 180, closed="right"): "90-180 days",
            pd.Interval(180, 365, closed="right"): "180-365 days",
            pd.Interval(365, 730, closed="right"): "365-730 days",
            pd.Interval(730, 999999999, closed="right"): "730+ days",
        }
    )
)


,total_cases,pct
case_duration,,
0-7 days,2352,0.021
7-28 days,8844,0.080
28-90 days,21964,0.199
90-180 days,27335,0.248
180-365 days,30153,0.274
365-730 days,12405,0.113
730+ days,7068,0.064


In [14]:
cd_df.groupby("state_name").case_duration.describe()


,count,mean,std,min,25%,50%,75%,max
state_name,,,,,,,,
California,10961.000,521.275,305.989,26.000,299.000,437.000,672.000,1951.000
Indiana,3163.000,29.009,27.398,0.000,13.000,23.000,37.000,389.000
Kansas,9105.000,213.001,401.861,0.000,69.000,127.000,208.000,4105.000
South Carolina,14290.000,62.379,54.661,0.000,28.000,53.000,81.000,1203.000
Texas,60683.000,332.816,450.520,29.000,133.000,208.000,323.000,4585.000
Washington,6364.000,55.863,55.484,0.000,22.000,43.000,69.000,1197.000
West Virginia,3770.000,33.124,42.204,0.000,8.000,20.000,40.000,444.000
Wyoming,2092.000,100.001,137.623,0.000,24.000,48.000,124.000,1114.000
